# PyCoMo Maximum Growth-Rate #
This tutorial show-cases how to find the maximum growth-rate of a community - across all possible abundance profiles.

The expected runtime for this notebook is less than 10 minutes.

In [1]:
from pathlib import Path
import sys
import os
import cobra
import math 
import time
import warnings

In [2]:
path_root = "../src"  # Change path according to your PyCoMo location
sys.path.append(str(path_root))
import pycomo

2024-07-08 11:44:32,852 - pycomo.helper.multiprocess - INFO - Multiprocess Logger initialized.
2024-07-08 11:44:32,854 - pycomo.pycomo_models - INFO - Logger initialized.


## Load the Toy Models ##
These are simple toy models of 2 organisms. In a community, they are expexted to grow at a maximum of 10/h, however one organism alone can grow as fast as 15/h. Both cases can be calculated.

In [3]:
# create two cobra models from sbml (change the path according to where the models are located)
toy1 = cobra.io.read_sbml_model("../data/use_case/toy_models/toy_1.xml")
toy2 = cobra.io.read_sbml_model("../data/use_case/toy_models/toy_2_2.xml")

# create Single Organism Models from cobra models
Toy1 = pycomo.SingleOrganismModel(toy1, "toy1")
Toy2 = pycomo.SingleOrganismModel(toy2, "toy2")

# create Community Model from Single Organism Models
C = pycomo.CommunityModel([Toy1, Toy2], name = "Toy_community")
# instantiate the model and set cplex as a solver
C.model.solver = "cplex"

No community model generated yet. Generating now:


Ignoring reaction 'EX_C_medium' since it already exists.


Generated community model.


## Compute maximum community growth rate

The method ```max_growth_rate``` calculates the overall maximum growth rate of the community, regardless of its composition.

In [4]:
C.max_growth_rate()

New round
lb: 0.0, ub: 1000.0, x: 500.0
Infeasible!
New round
lb: 0.0, ub: 500.0, x: 250.0
Infeasible!
New round
lb: 0.0, ub: 250.0, x: 125.0
Infeasible!
New round
lb: 0.0, ub: 125.0, x: 62.5
Infeasible!
New round
lb: 0.0, ub: 62.5, x: 31.25
Infeasible!
New round
lb: 0.0, ub: 31.25, x: 15.625
Infeasible!
New round
lb: 0.0, ub: 15.625, x: 7.8125
                                   reaction_id  min_flux  max_flux
toy1_fraction_reaction  toy1_fraction_reaction       0.0       1.0
toy2_fraction_reaction  toy2_fraction_reaction       0.0       1.0
lb_df: toy1_fraction_reaction    0
toy2_fraction_reaction    0
Name: min_flux, dtype: int64
R_df: toy1_fraction_reaction    1.0
toy2_fraction_reaction    1.0
dtype: float64
Delta: 1.0
Sum R_df: 2.0
ab_df: toy1_fraction_reaction    0.5
toy2_fraction_reaction    0.5
dtype: float64
fba results: 10.0
New round
lb: 10.0, ub: 15.625, x: 10.000002
                                   reaction_id  min_flux  max_flux
toy1_fraction_reaction  toy1_fraction_reac

15.0

```max_growth_rate``` has the following parameters:
* ```minimal_abundance``` (default = 0)
* ```return_abundances``` (default = False)
* ```sensitivity``` (default = 6)
* ```gurobi``` (default = False)

### Return Abundances
By setting the parameter ```return_abundances``` to ```True```, additional information about the feasible community compositions at the maximum growth rate is returned in the form of a pandas dataframe.

The minimal flux of x_fraction_reaction corresponds to the minimal abundance of member x in the community. Likewise, the maximal flux corresponds to the maximal abundance of member x.

In [5]:
C.max_growth_rate(return_abundances=True)

New round
lb: 0.0, ub: 1000.0, x: 500.0
Infeasible!
New round
lb: 0.0, ub: 500.0, x: 250.0
Infeasible!
New round
lb: 0.0, ub: 250.0, x: 125.0
Infeasible!
New round
lb: 0.0, ub: 125.0, x: 62.5
Infeasible!
New round
lb: 0.0, ub: 62.5, x: 31.25
Infeasible!
New round
lb: 0.0, ub: 31.25, x: 15.625
Infeasible!
New round
lb: 0.0, ub: 15.625, x: 7.8125
                                   reaction_id  min_flux  max_flux
toy1_fraction_reaction  toy1_fraction_reaction       0.0       1.0
toy2_fraction_reaction  toy2_fraction_reaction       0.0       1.0
lb_df: toy1_fraction_reaction    0
toy2_fraction_reaction    0
Name: min_flux, dtype: int64
R_df: toy1_fraction_reaction    1.0
toy2_fraction_reaction    1.0
dtype: float64
Delta: 1.0
Sum R_df: 2.0
ab_df: toy1_fraction_reaction    0.5
toy2_fraction_reaction    0.5
dtype: float64
fba results: 10.0
New round
lb: 10.0, ub: 15.625, x: 10.000002
                                   reaction_id  min_flux  max_flux
toy1_fraction_reaction  toy1_fraction_reac

,reaction_id,min_flux,max_flux
0,toy1_fraction_reaction,0.0,0.0
1,toy2_fraction_reaction,1.0,1.0
2,community_biomass,15.0,15.0


### Minimal abundance
Edge cases exist in which the highest growth rate of a community is not achieved by a combination of all the members. Instead, one member with an abundance of 100% may achieve the maximum growth rate. 

In order to exclude these cases, the parameter ```minimal_abundance``` must be set to a float greater than 0. This sets the minimal abundance of all members to the set float. Take care that the sum of the minimal abundances is not greater than 1, as this will result in an error.

In [6]:
C.max_growth_rate(minimal_abundance=0.1)

New round
lb: 0.0, ub: 1000.0, x: 500.0
Infeasible!
New round
lb: 0.0, ub: 500.0, x: 250.0
Infeasible!
New round
lb: 0.0, ub: 250.0, x: 125.0
Infeasible!
New round
lb: 0.0, ub: 125.0, x: 62.5
Infeasible!
New round
lb: 0.0, ub: 62.5, x: 31.25
Infeasible!
New round
lb: 0.0, ub: 31.25, x: 15.625
Infeasible!
New round
lb: 0.0, ub: 15.625, x: 7.8125
                                   reaction_id  min_flux  max_flux
toy1_fraction_reaction  toy1_fraction_reaction       0.1       0.9
toy2_fraction_reaction  toy2_fraction_reaction       0.1       0.9
lb_df: toy1_fraction_reaction    0.1
toy2_fraction_reaction    0.1
Name: min_flux, dtype: float64
R_df: toy1_fraction_reaction    0.8
toy2_fraction_reaction    0.8
dtype: float64
Delta: 0.8
Sum R_df: 1.6
ab_df: toy1_fraction_reaction    0.5
toy2_fraction_reaction    0.5
dtype: float64
fba results: 10.000000000000004
New round
lb: 10.000000000000004, ub: 15.625, x: 10.000002000000004
                                   reaction_id  min_flux  max_flux

10.0

In [7]:
# too large minimal_abundance value leads to an error
try:
    C.max_growth_rate(minimal_abundance=0.6)
except AssertionError:
    print("Error thrown!")
    
# The community has two members: 
# 2 * 0.6 = 1.2
# 1.2 > 1

Error thrown!


## Example: Biogas producing community ##
The example for this part will be a three member community published by Koch et al. 2019 (https://doi.org/10.1371/journal.pcbi.1006759). The three member organisms are representatives of functional guilds in a biogas community.

In [8]:
# create cobra models from sbml
dv = cobra.io.read_sbml_model("../data/use_case/koch/dv.xml")
mh = cobra.io.read_sbml_model("../data/use_case/koch/mh.xml")
mb = cobra.io.read_sbml_model("../data/use_case/koch/mb.xml")

# change infinite upper bounds to 1000
for model in [dv,mh,mb]:
    for reaction in model.reactions:
        if reaction.upper_bound == math.inf:
            reaction.upper_bound = 1000.
        if reaction.lower_bound == -math.inf:
            reaction.lower_bound = -1000.

# create Single Organism Models from cobra models
DV = pycomo.SingleOrganismModel(dv, "dv")
MH = pycomo.SingleOrganismModel(mh, "mh")
MB = pycomo.SingleOrganismModel(mb, "mb")

# create Community Model from Single Organism Models
C2 = pycomo.CommunityModel([DV, MH, MB], name = "dv_mh_mb_community")

# set solver to cplex 
C2.model.solver = "cplex"

# apply the same medium as in the paper by Koch et al.
medium = {
    'EX_CO2_EX_medium': 1000.0,
    'EX_Eth_EX_medium': 1000.0,
    'EX_BM_tot_medium': 1000.0
}
C2.medium = medium
C2.apply_medium()

# Formate and Hydrogen are not allowed to accumulate in the medium.
C2.model.reactions.get_by_id("EX_Form_EX_medium").upper_bound = 0.
C2.model.reactions.get_by_id("EX_H2_EX_medium").upper_bound = 0.


'3PG' is not a valid SBML 'SId'.
'2PG' is not a valid SBML 'SId'.
Adding exchange reaction EX_H2_EX with default bounds for boundary metabolite: H2_EX.
Adding exchange reaction EX_Ac_EX with default bounds for boundary metabolite: Ac_EX.
Adding exchange reaction EX_CO2_EX with default bounds for boundary metabolite: CO2_EX.
Adding exchange reaction EX_Form_EX with default bounds for boundary metabolite: Form_EX.
Adding exchange reaction EX_SO4_EX with default bounds for boundary metabolite: SO4_EX.
Adding exchange reaction EX_H2S_EX with default bounds for boundary metabolite: H2S_EX.
Adding exchange reaction EX_Eth_EX with default bounds for boundary metabolite: Eth_EX.
Adding exchange reaction EX_Lac_EX with default bounds for boundary metabolite: Lac_EX.
Adding exchange reaction EX_Pyr_EX with default bounds for boundary metabolite: Pyr_EX.
Adding exchange reaction EX_BM_tot with default bounds for boundary metabolite: BM_tot.
'2PG__PEP' is not a valid SBML 'SId'.
'3PG__2PG' is not 

No community model generated yet. Generating now:


Ignoring reaction 'EX_H2_EX_medium' since it already exists.
Ignoring reaction 'EX_CO2_EX_medium' since it already exists.
Ignoring reaction 'EX_Form_EX_medium' since it already exists.


Ignoring reaction 'EX_H2_EX_medium' since it already exists.
Ignoring reaction 'EX_CO2_EX_medium' since it already exists.
Ignoring reaction 'EX_CH4_EX_medium' since it already exists.
Ignoring reaction 'EX_Ac_EX_medium' since it already exists.
Ignoring reaction 'EX_BM_tot_medium' since it already exists.


Generated community model.


In [9]:
# calculate max growth rate of community
C2.max_growth_rate()

New round
lb: 0.0, ub: 1000.0, x: 500.0
Infeasible!
New round
lb: 0.0, ub: 500.0, x: 250.0
Infeasible!
New round
lb: 0.0, ub: 250.0, x: 125.0
Infeasible!
New round
lb: 0.0, ub: 125.0, x: 62.5
Infeasible!
New round
lb: 0.0, ub: 62.5, x: 31.25
Infeasible!
New round
lb: 0.0, ub: 31.25, x: 15.625
Infeasible!
New round
lb: 0.0, ub: 15.625, x: 7.8125
Infeasible!
New round
lb: 0.0, ub: 7.8125, x: 3.90625
Infeasible!
New round
lb: 0.0, ub: 3.90625, x: 1.953125
Infeasible!
New round
lb: 0.0, ub: 1.953125, x: 0.9765625
Infeasible!
New round
lb: 0.0, ub: 0.9765625, x: 0.48828125
Infeasible!
New round
lb: 0.0, ub: 0.48828125, x: 0.244140625
Infeasible!
New round
lb: 0.0, ub: 0.244140625, x: 0.1220703125
Infeasible!
New round
lb: 0.0, ub: 0.1220703125, x: 0.06103515625
Infeasible!
New round
lb: 0.0, ub: 0.06103515625, x: 0.030517578125
                               reaction_id  min_flux  max_flux
dv_fraction_reaction  dv_fraction_reaction  0.054405  0.440197
mh_fraction_reaction  mh_fraction_react

0.052128

In [10]:
# calculate abundance profile as well
C2.max_growth_rate(return_abundances=True)

New round
lb: 0.0, ub: 1000.0, x: 500.0
Infeasible!
New round
lb: 0.0, ub: 500.0, x: 250.0
Infeasible!
New round
lb: 0.0, ub: 250.0, x: 125.0
Infeasible!
New round
lb: 0.0, ub: 125.0, x: 62.5
Infeasible!
New round
lb: 0.0, ub: 62.5, x: 31.25
Infeasible!
New round
lb: 0.0, ub: 31.25, x: 15.625
Infeasible!
New round
lb: 0.0, ub: 15.625, x: 7.8125
Infeasible!
New round
lb: 0.0, ub: 7.8125, x: 3.90625
Infeasible!
New round
lb: 0.0, ub: 3.90625, x: 1.953125
Infeasible!
New round
lb: 0.0, ub: 1.953125, x: 0.9765625
Infeasible!
New round
lb: 0.0, ub: 0.9765625, x: 0.48828125
Infeasible!
New round
lb: 0.0, ub: 0.48828125, x: 0.244140625
Infeasible!
New round
lb: 0.0, ub: 0.244140625, x: 0.1220703125
Infeasible!
New round
lb: 0.0, ub: 0.1220703125, x: 0.06103515625
Infeasible!
New round
lb: 0.0, ub: 0.06103515625, x: 0.030517578125
                               reaction_id  min_flux  max_flux
dv_fraction_reaction  dv_fraction_reaction  0.054405  0.440197
mh_fraction_reaction  mh_fraction_react

,reaction_id,min_flux,max_flux
0,dv_fraction_reaction,0.074553,0.393590
1,mh_fraction_reaction,0.000000,0.681848
2,mb_fraction_reaction,0.000000,0.925447
3,community_biomass,0.052128,0.052128


### Sensitivity

The ```sensitivity``` parameter describes the amount of decimal places that should be calculated. The default is set to 6, as the cplex solver may return inaccurate results at a higher value.

In [11]:
# sensitivity = 4
C2.max_growth_rate(sensitivity=4)

New round
lb: 0.0, ub: 1000.0, x: 500.0
Infeasible!
New round
lb: 0.0, ub: 500.0, x: 250.0
Infeasible!
New round
lb: 0.0, ub: 250.0, x: 125.0
Infeasible!
New round
lb: 0.0, ub: 125.0, x: 62.5
Infeasible!
New round
lb: 0.0, ub: 62.5, x: 31.25
Infeasible!
New round
lb: 0.0, ub: 31.25, x: 15.625
Infeasible!
New round
lb: 0.0, ub: 15.625, x: 7.8125
Infeasible!
New round
lb: 0.0, ub: 7.8125, x: 3.90625
Infeasible!
New round
lb: 0.0, ub: 3.90625, x: 1.953125
Infeasible!
New round
lb: 0.0, ub: 1.953125, x: 0.9765625
Infeasible!
New round
lb: 0.0, ub: 0.9765625, x: 0.48828125
Infeasible!
New round
lb: 0.0, ub: 0.48828125, x: 0.244140625
Infeasible!
New round
lb: 0.0, ub: 0.244140625, x: 0.1220703125
Infeasible!
New round
lb: 0.0, ub: 0.1220703125, x: 0.06103515625
Infeasible!
New round
lb: 0.0, ub: 0.06103515625, x: 0.030517578125
                               reaction_id  min_flux  max_flux
dv_fraction_reaction  dv_fraction_reaction  0.054405  0.440197
mh_fraction_reaction  mh_fraction_react

0.0521

In [12]:
# sensitivity = 2
C2.max_growth_rate(sensitivity=2)

New round
lb: 0.0, ub: 1000.0, x: 500.0
Infeasible!
New round
lb: 0.0, ub: 500.0, x: 250.0
Infeasible!
New round
lb: 0.0, ub: 250.0, x: 125.0
Infeasible!
New round
lb: 0.0, ub: 125.0, x: 62.5
Infeasible!
New round
lb: 0.0, ub: 62.5, x: 31.25
Infeasible!
New round
lb: 0.0, ub: 31.25, x: 15.625
Infeasible!
New round
lb: 0.0, ub: 15.625, x: 7.8125
Infeasible!
New round
lb: 0.0, ub: 7.8125, x: 3.90625
Infeasible!
New round
lb: 0.0, ub: 3.90625, x: 1.953125
Infeasible!
New round
lb: 0.0, ub: 1.953125, x: 0.9765625
Infeasible!
New round
lb: 0.0, ub: 0.9765625, x: 0.48828125
Infeasible!
New round
lb: 0.0, ub: 0.48828125, x: 0.244140625
Infeasible!
New round
lb: 0.0, ub: 0.244140625, x: 0.1220703125
Infeasible!
New round
lb: 0.0, ub: 0.1220703125, x: 0.06103515625
Infeasible!
New round
lb: 0.0, ub: 0.06103515625, x: 0.030517578125
                               reaction_id  min_flux  max_flux
dv_fraction_reaction  dv_fraction_reaction  0.054405  0.440197
mh_fraction_reaction  mh_fraction_react

0.05